In [1]:
import copy
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os

from transformers import GPT2Tokenizer
from transformers import GPT2ForSequenceClassification

from sklearn.model_selection import train_test_split
from sklearn import metrics
from datasets import Dataset

/opt/homebrew/anaconda3/envs/CS6120/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device('cude')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [3]:
from visualize_attention_heatmap import plot_head0_heatmap as plot_heatmap

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

## Adds the tokenizer to the model (Un comment these two only if ./model_with_tokeinzer folder doesn't have tokenizer_config.json)
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.save_pretrained('model_with_tokenizer')

main_model = GPT2ForSequenceClassification.from_pretrained("./model")

sm = nn.Softmax()

mapping = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

In [4]:
for n, p in main_model.named_parameters():
    print(f"{n}: {p.shape}")


transformer.wte.weight: torch.Size([50257, 768])
transformer.wpe.weight: torch.Size([1024, 768])
transformer.h.0.ln_1.weight: torch.Size([768])
transformer.h.0.ln_1.bias: torch.Size([768])
transformer.h.0.attn.c_attn.weight: torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias: torch.Size([2304])
transformer.h.0.attn.c_proj.weight: torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias: torch.Size([768])
transformer.h.0.ln_2.weight: torch.Size([768])
transformer.h.0.ln_2.bias: torch.Size([768])
transformer.h.0.mlp.c_fc.weight: torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias: torch.Size([3072])
transformer.h.0.mlp.c_proj.weight: torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias: torch.Size([768])
transformer.h.1.ln_1.weight: torch.Size([768])
transformer.h.1.ln_1.bias: torch.Size([768])
transformer.h.1.attn.c_attn.weight: torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias: torch.Size([2304])
transformer.h.1.attn.c_proj.weight: torch.Size([768, 768])
transformer.h.

In [5]:
def pass_text_to_model(inputs, model):

    ## collect prob vectors
    with torch.no_grad():
        logits = model(**inputs).logits
    softmax_probs = sm(logits)
    return softmax_probs

def token_analysis(text_1, text_2):
    inputs_1 = tokenizer(text_1, return_tensors = 'pt')
    inputs_2 = tokenizer(text_2, return_tensors = 'pt')

    probability_vector_1 = pass_text_to_model(inputs_1, main_model)
    probability_vector_2 = pass_text_to_model(inputs_2, main_model)

    print(probability_vector_1, f"Label = {np.argmax(probability_vector_1.tolist()[0])}")
    print(probability_vector_2, f"Label = {np.argmax(probability_vector_2.tolist()[0])}")

## Flipping Gender

In [6]:
text_1 = "i did enjoy feeling like i could just relax with my sweet little bundle amp drift into slumber" 
text_2 = "i did enjoy feeling like i could just relax with my pleasant small bundle amp drift into slumber"

token_analysis(text_1, text_2)

text_1 = "i dont want to do those things when i feel crappy" 
text_2 = "i dont desire to do those things when i feel crappy"

token_analysis(text_1, text_2)

text_1 = "i feel like i am all out of funny"
text_2 = "i feel like i am all out of hilarious"

token_analysis(text_1, text_2)

tensor([[3.2963e-04, 5.2088e-01, 4.7775e-01, 4.5614e-04, 3.4623e-04, 2.3097e-04]]) Label = 1
tensor([[7.6754e-07, 1.0000e+00, 1.2015e-06, 9.3420e-07, 5.3941e-07, 1.3400e-08]]) Label = 1
tensor([[1.0000e+00, 1.1355e-06, 3.4172e-07, 5.6469e-07, 4.2074e-07, 2.7909e-07]]) Label = 0
tensor([[1.0000e+00, 1.1330e-06, 3.4841e-07, 5.7485e-07, 4.2025e-07, 2.7389e-07]]) Label = 0
tensor([[5.0603e-08, 1.0684e-06, 2.3182e-06, 4.3599e-07, 1.1583e-05, 9.9998e-01]]) Label = 5
tensor([[1.3948e-06, 7.2539e-01, 1.1019e-06, 4.5965e-06, 1.3785e-06, 2.7460e-01]]) Label = 1


/opt/homebrew/anaconda3/envs/CS6120/lib/python3.10/site-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


## Try your own examples!

In [ ]:
# Try your own text!

text1 = 'Your text here'
text2 = 'Your text here'
token_analysis(text_1, text_2)